In [18]:
from sklearn.datasets import make_classification
from sklearn import preprocessing
import pandas as pd
import numpy as np
import re
import bk_random
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
data = pd.read_csv('data/GTPD_bike_data.csv')
drop_cols = [x for x in data.columns.values if x.startswith('Unnamed')]
bad_cols = [
    'LocationDirectional',
    'CaseDisposition',
    'LocationLatitude',
    'LocationLongitude',
    'LocationStreet'
]
drop_cols += bad_cols
data.drop(drop_cols, axis=1, inplace=True)
data = data.fillna('...')
data.head()

,IncidentFromDate,IncidentFromTime,IncidentToDate,IncidentToTime,CaseStatus,LocationCode,PatrolZone,LocationLandmark,LocationStreetNumber
0,01/13/2013,18:00:00,01/14/2013,13:00:00,Inactive,NONCAM,Z2,Theta XI Fraternity 329A Z2,777
1,02/06/2013,23:00:00,02/07/2013,12:00:00,Inactive,ONCAM,Z2,Delta Chi Fraternity,170
2,02/14/2013,11:00:00,02/21/2013,09:30:00,Inactive,ONCAM,Z2,Harrison Residence Hall 14 Z2,660
3,02/26/2013,21:00:00,02/27/2013,08:30:00,Inactive,ONCAM,Z4,Hefner Residence Hall,510
4,02/27/2013,22:00:00,03/04/2013,08:00:00,Inactive,ONCAM,Z2,Towers Residence Hall,112


In [3]:
import matplotlib.pyplot as plt
pd.value_counts(data['LocationStreetNumber'])[:50].plot.bar()
plt.title('LocationStreetNumber')
plt.show()

In [4]:
'''
TODO:

**msg me if you have questions**

1. convert incidentfromdates to months - disregard year - one-hot encode
2. convert incidenttodate to months - disregard year - one-hot-encode
3. tokenize incidentfromtime && incidenttotime- one-hot encode
4. one-hot encode casestatus
5. one-hot encode locationcode
6. clean out locationlandmark
    - remove all: numbers, (SST), symbols, etc
    - one-hot encode
7. one-hot encode locationstreetnumber
8. one-hot encode locationstreet
'''

'\nTODO:\n\n**msg me if you have questions**\n\n1. convert incidentfromdates to months - disregard year - one-hot encode\n2. convert incidenttodate to months - disregard year - one-hot-encode\n3. tokenize incidentfromtime && incidenttotime- one-hot encode\n4. one-hot encode casestatus\n5. one-hot encode locationcode\n6. clean out locationlandmark\n    - remove all: numbers, (SST), symbols, etc\n    - one-hot encode\n7. one-hot encode locationstreetnumber\n8. one-hot encode locationstreet\n'

In [5]:
# Clean from and to date data
def return_month(data):
    data = data.split('/')[0]
    if data[0] == '0':
        data = data[1:]
    return data

data['IncidentFromDate'] = data.apply(lambda row: return_month(row['IncidentFromDate']), axis=1)
data['IncidentToDate'] = data.apply(lambda row: return_month(row['IncidentToDate']), axis=1)

In [6]:
# Clean to and from time
def return_hour(data):
    data = data.split(':')[0]
    if data[0] == '0':
        data = data[1:]
    return data

data['IncidentFromTime'] = data.apply(lambda row: return_hour(row['IncidentFromTime']), axis=1)
data['IncidentToTime'] = data.apply(lambda row: return_hour(row['IncidentToTime']), axis=1)

In [7]:
# Clean location landmark data, along with case status and location street
data['LocationLandmark'] = data['LocationLandmark'].str.replace(r"\(.*\)","")
data['LocationLandmark'] = data['LocationLandmark'].str.replace(r"\d+", "")
data['LocationLandmark'] = data['LocationLandmark'].str.replace(' Z', '')
data['LocationLandmark'] = data['LocationLandmark'].str.upper()
data['CaseStatus'] = data['CaseStatus'].str.upper()
data['LocationStreetNumber'] = data['LocationStreetNumber'].astype(str)
data.head()

,IncidentFromDate,IncidentFromTime,IncidentToDate,IncidentToTime,CaseStatus,LocationCode,PatrolZone,LocationLandmark,LocationStreetNumber
0,1,18,1,13,INACTIVE,NONCAM,Z2,THETA XI FRATERNITY A,777.0
1,2,23,2,12,INACTIVE,ONCAM,Z2,DELTA CHI FRATERNITY,170.0
2,2,11,2,9,INACTIVE,ONCAM,Z2,HARRISON RESIDENCE HALL,660.0
3,2,21,2,8,INACTIVE,ONCAM,Z4,HEFNER RESIDENCE HALL,510.0
4,2,22,3,8,INACTIVE,ONCAM,Z2,TOWERS RESIDENCE HALL,112.0


In [8]:
one_hot_data = pd.get_dummies(data)

In [20]:
one_hot_data.head(20)

,IncidentFromDate_1,IncidentFromDate_10,IncidentFromDate_11,IncidentFromDate_12,IncidentFromDate_2,IncidentFromDate_3,IncidentFromDate_4,IncidentFromDate_5,IncidentFromDate_6,IncidentFromDate_7,IncidentFromDate_8,IncidentFromDate_9,IncidentFromTime_,IncidentFromTime_0,IncidentFromTime_1,IncidentFromTime_10,IncidentFromTime_11,IncidentFromTime_12,IncidentFromTime_13,IncidentFromTime_14,IncidentFromTime_15,IncidentFromTime_16,IncidentFromTime_17,IncidentFromTime_18,IncidentFromTime_19,IncidentFromTime_2,IncidentFromTime_20,IncidentFromTime_21,IncidentFromTime_22,IncidentFromTime_23,IncidentFromTime_3,IncidentFromTime_4,IncidentFromTime_5,IncidentFromTime_6,IncidentFromTime_7,IncidentFromTime_8,IncidentFromTime_9,IncidentToDate_...,IncidentToDate_1,IncidentToDate_10,IncidentToDate_11,IncidentToDate_12,IncidentToDate_2,IncidentToDate_3,IncidentToDate_4,IncidentToDate_5,IncidentToDate_6,IncidentToDate_7,IncidentToDate_8,IncidentToDate_9,IncidentToTime_,IncidentToTime_...,IncidentToTime_0,IncidentToTime_1,IncidentToTime_10,IncidentToTime_11,IncidentToTime_12,IncidentToTime_13,IncidentToTime_14,IncidentToTime_15,IncidentToTime_16,IncidentToTime_17,IncidentToTime_18,IncidentToTime_19,IncidentToTime_2,IncidentToTime_20,IncidentToTime_21,IncidentToTime_22,IncidentToTime_23,IncidentToTime_3,IncidentToTime_4,IncidentToTime_5,IncidentToTime_6,IncidentToTime_7,IncidentToTime_8,IncidentToTime_9,CaseStatus_CLOSED/CLEARED,CaseStatus_FURTHER INVESTIGATION,CaseStatus_INACTIVE,LocationCode_NONCAM,LocationCode_NONCLERY,LocationCode_ONCAM,LocationCode_ONCAMRES,LocationCode_PUB,PatrolZone_...,PatrolZone_OFFCAM,PatrolZone_Z1,PatrolZone_Z2,PatrolZone_Z3,PatrolZone_Z4,"LocationLandmark_ TECHWOOD DRIVE, NW",LocationLandmark_...,LocationLandmark_ALPHA EPSILON PI FRATERNITY,LocationLandmark_ALUMNI-FACULTY HOUSE,LocationLandmark_ARCHITECTURE CARD LOT,LocationLandmark_ARMSTRONG RESIDENCE HALL,LocationLandmark_BARNES AND NOBLE BOOKSTORE,LocationLandmark_BARREL HOUSE,LocationLandmark_BETA THETA PI FRATERNITY,LocationLandmark_BETWEEN LOVE BUILDING & INSTRUCTIONAL CENTER,LocationLandmark_BOGGS BUILDING,LocationLandmark_BRITTAIN DINING HALL,LocationLandmark_BRITTAIN DINING HALL,LocationLandmark_BRITTAN DINNING HALL,LocationLandmark_BROWN RESIDENCE HALL,LocationLandmark_BURGER BOWL FIELD,LocationLandmark_CALDWELL RESIDENCE HALL,LocationLandmark_CALDWELL RESIDENCE HALL,LocationLandmark_CAMPUS CHRISTIAN FELLOWSHIP,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER PARKING DECK,LocationLandmark_CARBON-NEUTRAL ENERGY SOLUTIONS LAB,LocationLandmark_CATHOLIC CENTER,LocationLandmark_CENTENNIAL RESEARCH BUILDING,LocationLandmark_CENTER FOR ADVANCED BRAIN IMAGING,LocationLandmark_CENTER STREET APARTMENT,LocationLandmark_CENTER STREET APARTMENTS,LocationLandmark_CENTER STREET APARTMENTS,LocationLandmark_CHERRY EMERSON,LocationLandmark_CHERRY EMERSON BUILDING,LocationLandmark_CHRISTIAN CAMPUS FELLOWSHIP,LocationLandmark_CHRISTIAN CAMPUS FELLOWSHIP NORTH,LocationLandmark_CLOUDMAN RESIDENCE HALL,LocationLandmark_CLOUDMAN RESIDENCE HALL,LocationLandmark_CLOUGH BUILDING,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING CENTER,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING CENTER,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING COMMONS,LocationLandmark_COLLEGE OF ARCHITECTURE EAST,LocationLandmark_COLLEGE OF ARCHITECTURE EAST,LocationLandmark_COLLEGE OF ARCHITECTURE- EAST WING,LocationLandmark_COLLEGE OF ARCHITECTURE-WEST WING,LocationLandmark_COLLEGE OF ARCHITECTURE-WEST WING,LocationLandmark_COLLEGE OF COMPUTING,LocationLandmark_COLLEGE OF COMPUTING BUILDING,LocationLandmark_CRECINE RESIDENCE HAL,LocationLandmark_D.M. SMITH BUILDING,LocationLandmark_DELTA CHI,LocationLandmark_DELTA CHI FRATERNITY,LocationLandmark_DELTA TAU DELTA FRATERNITY,LocationLandmark_EDGE ATHLETIC CENTER,LocationLandmark_EIGHTH STREET APARTMENTS,LocationLandmark_EIGHTH STREET APARTMENTS,Locatio

In [30]:
# def generate
## takes as input a dictionary of the title:value
## ex: {'IncidentFromDate_1': 41, 'IncidentFromDate_10': 36}
## normalize the values - now that probability is the probability that that will happen

def create_map(data):
    titles = data.columns.values
    mapping = {}
    norm = len(data)
    for title in titles:
        val = sum(data[title].tolist()) / norm
        mapping[title] = val
    return mapping
mapping = create_map(one_hot_data)
mapping

{'CaseStatus_CLOSED/CLEARED': 0.15753424657534246,
 'CaseStatus_FURTHER INVESTIGATION': 0.05821917808219178,
 'CaseStatus_INACTIVE': 0.7842465753424658,
 'IncidentFromDate_1': 0.06506849315068493,
 'IncidentFromDate_10': 0.1095890410958904,
 'IncidentFromDate_11': 0.1232876712328767,
 'IncidentFromDate_12': 0.05821917808219178,
 'IncidentFromDate_2': 0.0821917808219178,
 'IncidentFromDate_3': 0.07534246575342465,
 'IncidentFromDate_4': 0.07534246575342465,
 'IncidentFromDate_5': 0.07876712328767123,
 'IncidentFromDate_6': 0.05136986301369863,
 'IncidentFromDate_7': 0.030821917808219176,
 'IncidentFromDate_8': 0.10273972602739725,
 'IncidentFromDate_9': 0.14726027397260275,
 'IncidentFromTime_': 0.003424657534246575,
 'IncidentFromTime_0': 0.010273972602739725,
 'IncidentFromTime_1': 0.003424657534246575,
 'IncidentFromTime_10': 0.05821917808219178,
 'IncidentFromTime_11': 0.05821917808219178,
 'IncidentFromTime_12': 0.04452054794520548,
 'IncidentFromTime_13': 0.06164383561643835,
 'In

In [69]:
def synthesizer(orig_df, mapping, titles, iteration):
    ## mapping ex: {'IncidentFromDate_1': 0.03, 'IncidentFromDate_10': 0.2}
    ## titles ex: ['IncidentFromDate_1', 'IncidentFromDate_10' in order of how you want the df to be formatted]
    ## iteration : # of data needed
    df = []
    start_len = len(orig_df)
    for i in range(iteration):
        row = []
        for title in titles:
            val = mapping[title]
            synthetic = int(np.random.choice([1,0],1,p=[val,1-val])[0])
            row.append(synthetic)
        df.append(row)
    df = np.asarray(df)
    orig_df = orig_df.append(pd.DataFrame(df, columns=titles), ignore_index=True)
    print ('Finished generating synthetic data. Added ' +str(iteration)+' lines of data.')
    return orig_df

test = one_hot_data.head()
ret_df = synthesizer(test, create_map(one_hot_data), one_hot_data.columns.values, 5)
ret_df

Finished generating synthetic data. Added 5 lines of data.


,IncidentFromDate_1,IncidentFromDate_10,IncidentFromDate_11,IncidentFromDate_12,IncidentFromDate_2,IncidentFromDate_3,IncidentFromDate_4,IncidentFromDate_5,IncidentFromDate_6,IncidentFromDate_7,IncidentFromDate_8,IncidentFromDate_9,IncidentFromTime_,IncidentFromTime_0,IncidentFromTime_1,IncidentFromTime_10,IncidentFromTime_11,IncidentFromTime_12,IncidentFromTime_13,IncidentFromTime_14,IncidentFromTime_15,IncidentFromTime_16,IncidentFromTime_17,IncidentFromTime_18,IncidentFromTime_19,IncidentFromTime_2,IncidentFromTime_20,IncidentFromTime_21,IncidentFromTime_22,IncidentFromTime_23,IncidentFromTime_3,IncidentFromTime_4,IncidentFromTime_5,IncidentFromTime_6,IncidentFromTime_7,IncidentFromTime_8,IncidentFromTime_9,IncidentToDate_...,IncidentToDate_1,IncidentToDate_10,IncidentToDate_11,IncidentToDate_12,IncidentToDate_2,IncidentToDate_3,IncidentToDate_4,IncidentToDate_5,IncidentToDate_6,IncidentToDate_7,IncidentToDate_8,IncidentToDate_9,IncidentToTime_,IncidentToTime_...,IncidentToTime_0,IncidentToTime_1,IncidentToTime_10,IncidentToTime_11,IncidentToTime_12,IncidentToTime_13,IncidentToTime_14,IncidentToTime_15,IncidentToTime_16,IncidentToTime_17,IncidentToTime_18,IncidentToTime_19,IncidentToTime_2,IncidentToTime_20,IncidentToTime_21,IncidentToTime_22,IncidentToTime_23,IncidentToTime_3,IncidentToTime_4,IncidentToTime_5,IncidentToTime_6,IncidentToTime_7,IncidentToTime_8,IncidentToTime_9,CaseStatus_CLOSED/CLEARED,CaseStatus_FURTHER INVESTIGATION,CaseStatus_INACTIVE,LocationCode_NONCAM,LocationCode_NONCLERY,LocationCode_ONCAM,LocationCode_ONCAMRES,LocationCode_PUB,PatrolZone_...,PatrolZone_OFFCAM,PatrolZone_Z1,PatrolZone_Z2,PatrolZone_Z3,PatrolZone_Z4,"LocationLandmark_ TECHWOOD DRIVE, NW",LocationLandmark_...,LocationLandmark_ALPHA EPSILON PI FRATERNITY,LocationLandmark_ALUMNI-FACULTY HOUSE,LocationLandmark_ARCHITECTURE CARD LOT,LocationLandmark_ARMSTRONG RESIDENCE HALL,LocationLandmark_BARNES AND NOBLE BOOKSTORE,LocationLandmark_BARREL HOUSE,LocationLandmark_BETA THETA PI FRATERNITY,LocationLandmark_BETWEEN LOVE BUILDING & INSTRUCTIONAL CENTER,LocationLandmark_BOGGS BUILDING,LocationLandmark_BRITTAIN DINING HALL,LocationLandmark_BRITTAIN DINING HALL,LocationLandmark_BRITTAN DINNING HALL,LocationLandmark_BROWN RESIDENCE HALL,LocationLandmark_BURGER BOWL FIELD,LocationLandmark_CALDWELL RESIDENCE HALL,LocationLandmark_CALDWELL RESIDENCE HALL,LocationLandmark_CAMPUS CHRISTIAN FELLOWSHIP,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER,LocationLandmark_CAMPUS RECREATION CENTER PARKING DECK,LocationLandmark_CARBON-NEUTRAL ENERGY SOLUTIONS LAB,LocationLandmark_CATHOLIC CENTER,LocationLandmark_CENTENNIAL RESEARCH BUILDING,LocationLandmark_CENTER FOR ADVANCED BRAIN IMAGING,LocationLandmark_CENTER STREET APARTMENT,LocationLandmark_CENTER STREET APARTMENTS,LocationLandmark_CENTER STREET APARTMENTS,LocationLandmark_CHERRY EMERSON,LocationLandmark_CHERRY EMERSON BUILDING,LocationLandmark_CHRISTIAN CAMPUS FELLOWSHIP,LocationLandmark_CHRISTIAN CAMPUS FELLOWSHIP NORTH,LocationLandmark_CLOUDMAN RESIDENCE HALL,LocationLandmark_CLOUDMAN RESIDENCE HALL,LocationLandmark_CLOUGH BUILDING,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING CENTER,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING CENTER,LocationLandmark_CLOUGH UNDERGRADUATE LEARNING COMMONS,LocationLandmark_COLLEGE OF ARCHITECTURE EAST,LocationLandmark_COLLEGE OF ARCHITECTURE EAST,LocationLandmark_COLLEGE OF ARCHITECTURE- EAST WING,LocationLandmark_COLLEGE OF ARCHITECTURE-WEST WING,LocationLandmark_COLLEGE OF ARCHITECTURE-WEST WING,LocationLandmark_COLLEGE OF COMPUTING,LocationLandmark_COLLEGE OF COMPUTING BUILDING,LocationLandmark_CRECINE RESIDENCE HAL,LocationLandmark_D.M. SMITH BUILDING,LocationLandmark_DELTA CHI,LocationLandmark_DELTA CHI FRATERNITY,LocationLandmark_DELTA TAU DELTA FRATERNITY,LocationLandmark_EDGE ATHLETIC CENTER,LocationLandmark_EIGHTH STREET APARTMENTS,LocationLandmark_EIGHTH STREET APARTMENTS,Locatio